# Epitome of Salt: Sunset Show Notes 
## Author: Kevin Stull
We are going to see how we did at rating the BLB cards before they came out on arena. 

## Guests
TODO

## DataSet
Link: TODO

Last updated: TODO

In [1]:
# Define raters and data file
raters = ['Kevin', 'Nate'] # TODO
data = 'OTJ_tier_list.csv' # TODO

In [17]:
import pandas as pd

# Function to read rater ratings
def read_ratings(rater):
    return pd.read_csv(f'data/{rater}.csv')

# Initialize eos DataFrame
eos = pd.DataFrame(columns=['Name', 'Color', 'Rarity'])

# Define the tier to numeric value mapping
tier_num = {'G': 0., 'D-': 1., 'D': 2., 'D+': 3., 
            'C-': 4., 'C': 5., 'C+': 6., 'B-': 7., 
            'B': 8., 'B+': 9., 'A-': 10., 'A': 11., 
            'A+': 12., 'F':0.}

# Read ratings for each rater and merge into eos DataFrame
is_first = True
for rater in raters:
    rater_data = read_ratings(rater)
    if is_first:
        eos['Name'] = rater_data['Name']
        eos['Color'] = rater_data['Color']
        eos['Rarity'] = rater_data['Rarity']
        is_first = False
    eos[rater] = rater_data['Rating']
    eos[f'{rater}_num'] = eos[rater].map(tier_num)

# Drop rows with any missing values
eos = eos.dropna().copy()

# Read the master tier list
master_list = pd.read_csv(f'data/{data}')

# Match names with master tier list and add tiers to eos
eos['Tier'] = eos['Name'].map(dict(zip(master_list['Name'], master_list['Tier'])))
eos['Tier_num'] = eos['Tier'].map(tier_num)

# Dropped entries due to insufficient data 
dropped_entries_initial = eos[eos.isna().any(axis=1)]
print(f"Initial entries to be dropped (due to missing data): {len(dropped_entries_initial)}")
print(dropped_entries_initial)

# Drop rows with missing Tier information
eos = eos.dropna().copy()

# Calculate rating errors for each rater
for rater in raters:
    eos[f'{rater}_error'] = abs(eos['Tier_num'] - eos[f'{rater}_num'])

# The final DataFrame
eos.to_csv('data/data.csv', index=False)
eos

Initial entries to be dropped (due to missing data): 4
                        Name Color Rarity Kevin  Kevin_num Nate  Nate_num  \
81     Neutralize the Guards     B      U    C-        4.0    D       2.0   
103  Caught in the Crossfire     R      U     D        2.0   D+       3.0   
106          Magebane Lizard     R      U    D+        3.0   D+       3.0   
108         Return the Favor     R      U    B+        9.0    D       2.0   

    Tier  Tier_num  
81    SB       NaN  
103   SB       NaN  
106   SB       NaN  
108   SB       NaN  


,Name,Color,Rarity,Kevin,Kevin_num,Nate,Nate_num,Tier,Tier_num,Kevin_error,Nate_error
0,Armored Armadillo,W,C,D,2.0,D+,3.0,D,2.0,0.0,1.0
1,Steer Clear,W,C,C-,4.0,C-,4.0,D+,3.0,1.0,1.0
2,Eriette's Lullaby,W,C,C-,4.0,C-,4.0,D+,3.0,1.0,1.0
3,Outlaw Medic,W,C,C,5.0,C+,6.0,C+,6.0,1.0,0.0
4,Sterling Keykeeper,W,C,C-,4.0,C-,4.0,C,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
266,Blooming Marsh,L,R,B,8.0,C-,4.0,C+,6.0,2.0,2.0
267,Botanical Sanctum,L,R,C+,6.0,C-,4.0,C+,6.0,0.0,2.0
268,Concealed Courtyard,L,R,B-,7.0,C-,4.0,C+,6.0,1.0,2.0
269,Inspiring Vantage,L,R,B,8.0,C-,4.0,C+,6.0,2.0,2.0


In [18]:
# See who's predictions did the best 
for rater in raters:
    error = eos[f'{rater}_error'].sum()
    print(f'{rater} error: {error}')

Kevin error: 467.0
Nate error: 523.0


In [19]:
# biggest misses
num = 10
for rater in raters:
    temp = eos.copy()
    temp = temp.sort_values(by=f'{rater}_error', ascending=False)
    print(temp[['Name', 'Tier', rater]][0:num])

                           Name Tier Kevin
203        The Key to the Vault    F     B
219           Great Train Heist   C-     A
254  Obeka, Splitter of Seconds    F    B-
210          Insatiable Avarice   D-     B
216            Pitiless Carnage   D-    B-
241      Wylie Duke, Atiin Hero   A-    C-
204   Fblthp, Lost on the Range    D     B
207             Jace Reawakened    D     B
15               Getaway Glamer    B     D
211          Tinybones Joins Up   D-    C+
                          Name Tier Nate
209                Double Down   A-   D-
204  Fblthp, Lost on the Range    D   A-
207            Jace Reawakened    D   A-
210         Insatiable Avarice   D-   B-
189              Bucolic Ranch   B-   D-
222         Slickshot Show-Off    D    B
256         Riku of Many Paths   D+   B+
86             Hollow Marauder   A-   C-
164         Make Your Own Luck    B    D
154         Congregation Gryff   A-    C


In [20]:
# hottest takes
num = 10
for rater in raters:
    temp = eos.copy()
    other_raters = [x for x in raters if x != rater]
    temp[f'{rater}_hot'] = abs(((temp[[f'{x}_num' for x in other_raters]].sum(axis=1)) / (len(other_raters))) - temp[f'{rater}_num'])
    temp = temp.sort_values(by=f'{rater}_hot', ascending=False)
    top_num_cols = ['Name', 'Tier'] + [f'{x}' for x in raters]
    top_num = temp[top_num_cols].head(num)
    print(f'Top {num} hottest takes for {rater}:')
    print(top_num)

Top 10 hottest takes for Kevin:
                     Name Tier Kevin Nate
209           Double Down   A-     B   D-
256    Riku of Many Paths   D+    D+   B+
197          One Last Job   B-    B-    D
203  The Key to the Vault    F     B   D+
150         At Knifepoint    D     D   B-
141  Rise of the Varmints   C+     B   D+
219     Great Train Heist   C-     A   C+
16        Nurturing Pixie   D+    D+   B-
82            Rakish Crew   C+     D   C+
202   Duelist of the Mind   B+     C   B+
Top 10 hottest takes for Nate:
                     Name Tier Kevin Nate
209           Double Down   A-     B   D-
256    Riku of Many Paths   D+    D+   B+
197          One Last Job   B-    B-    D
203  The Key to the Vault    F     B   D+
150         At Knifepoint    D     D   B-
141  Rise of the Varmints   C+     B   D+
219     Great Train Heist   C-     A   C+
16        Nurturing Pixie   D+    D+   B-
82            Rakish Crew   C+     D   C+
202   Duelist of the Mind   B+     C   B+


In [21]:
# Calculate optimism or pessimism metric for each rater
for rater in raters:
    total_difference = (eos[f'{rater}_num'] - eos['Tier_num']).sum()
    if total_difference > 0:
        print(f'{rater} tends to be optimistic by {total_difference} points.')
    elif total_difference < 0:
        print(f'{rater} tends to be pessimistic by {-total_difference} points.')
    else:
        print(f'{rater} has no overall bias.')

Kevin tends to be optimistic by 79.0 points.
Nate tends to be pessimistic by 37.0 points.
